In [1]:
import numpy as np
import pandas as pd
import os, pickle
from tqdm.notebook import tqdm

# In-Sample

In [2]:
# start insample logs dataframe? "err" = error (unscaled, no abs. value)
cols = ["model", "lmbda", "rho", "alpha", "dobs", "seed", 
        "betahat", "betahat_LB", "betahat_UB", "betahat_err",
        "rhohat", "rhohat_LB", "rhohat_UB", "rhohat_err",
        "sigmahat", "sigmahat_LB", "sigmahat_UB", "sigmahat_err",
        "X_rmse", "Y_rmse", "Z_rmse"]

# instantiate our dataframe
insample_logs, t_max = pd.DataFrame(data=None, columns=cols), 8.0

########

# in-sample logs first (MAGI) - theta pt estim, interval estim, covgs, theta-AEs, {X,Y,Z}-RMSEs
model = "magi"

# what are our relevant magi files?
fnames = sorted([f for f in os.listdir("results/magi") if "rho" in f])

# go thru each file + load in our .pickle, analyze the results
for fname in tqdm(fnames):
    
    # create our row for the dataframe
    rho, alpha, dobs, _, seed = [float(s.split("=")[1]) for s in fname.replace(".pickle", "").split("_")]
    dobs, seed = int(dobs), int(seed)
    row = [model, np.nan, rho, alpha, dobs, seed]
    
    # open up the pickle file
    with open(f"results/magi/{fname}", "rb") as file:
        results = pickle.load(file)
        
    # get our param estimates
    betahat, rhohat, sigmahat = results["thetas_samps"].mean(axis=0)
    betahat_LB, rhohat_LB, sigmahat_LB = np.quantile(a=results["thetas_samps"], q=0.025, axis=0)
    betahat_UB, rhohat_UB, sigmahat_UB = np.quantile(a=results["thetas_samps"], q=0.975, axis=0)
    
    # compute our errors
    betahat_err, rhohat_err, sigmahat_err = betahat - (8/3), rhohat - rho, sigmahat - 10.0
    
    # get our ground truth
    truth = pd.read_csv(f"data/LORENZ_rho={rho}_alpha={alpha}_seed={seed}.csv").query(f"t <= {t_max}")
    truth = truth.iloc[::int((truth.index.shape[0] - 1) / (dobs * t_max))]\
    [["t", "X_true", "Y_true", "Z_true"]].set_index("t")
    truth.index = np.round(truth.index, 5)
    
    # get our predictions for ts_obs ONLY!
    preds = pd.DataFrame(data=np.hstack([results["I"], results["X_samps"].mean(axis=0)]), 
                         columns=["t", "X", "Y", "Z"]).set_index("t")
    preds.index = np.round(preds.index, 5); preds = preds.loc[truth.index]
    
    # compute our RMSE on the ts_obs datapoints only!
    X_rmse, Y_rmse, Z_rmse = np.sqrt(((preds.values - truth.values) ** 2).mean(axis=0))
    
    # assemble our row + add to our dataframe
    row += [betahat, betahat_LB, betahat_UB, betahat_err, 
            rhohat, rhohat_LB, rhohat_UB, rhohat_err,
            sigmahat, sigmahat_LB, sigmahat_UB, sigmahat_err,
            X_rmse, Y_rmse, Z_rmse]
    insample_logs.loc[len(insample_logs.index)] = row
    
########

# repeat the process for PINN
model = "pinn"

# what are our relevant PINN files?
fnames = sorted([f for f in os.listdir("results/pinn") if "rho" in f])

# go thru each folder + load in our .csvs as called for
for fname in tqdm(fnames):
    
    # create our row for the dataframe
    lmbda, rho, alpha, dobs, seed = [float(s.split("=")[1]) for s in fname.split("_")]
    dobs, seed = int(dobs), int(seed)
    row = [model, lmbda, rho, alpha, dobs, seed]
    
    # get our param estimates and compute our errors
    betahat, rhohat, sigmahat = pd.read_csv(f"results/pinn/{fname}/theta_hats.csv").values[0]
    betahat_LB, rhohat_LB, sigmahat_LB = np.nan, np.nan, np.nan
    betahat_UB, rhohat_UB, sigmahat_UB = np.nan, np.nan, np.nan
    
    # compute our errors
    betahat_err, rhohat_err, sigmahat_err = betahat - (8/3), rhohat - rho, sigmahat - 10.0
    
    # get our ground truth
    truth = pd.read_csv(f"data/LORENZ_rho={rho}_alpha={alpha}_seed={seed}.csv").query(f"t <= {t_max}")
    truth = truth.iloc[::int((truth.index.shape[0] - 1) / (dobs * t_max))]\
    [["t", "X_true", "Y_true", "Z_true"]].set_index("t")
    truth.index = np.round(truth.index, 5)
    
    # get our predictions for ts_obs ONLY!
    preds = pd.read_csv(f"results/pinn/{fname}/in-sample_preds.csv").set_index("t")
    
    # compute our RMSE on the ts_obs datapoints only!
    X_rmse, Y_rmse, Z_rmse = np.sqrt(((preds.values - truth.values) ** 2).mean(axis=0))
    
    # assemble our row + add to our dataframe
    row += [betahat, betahat_LB, betahat_UB, betahat_err, 
            rhohat, rhohat_LB, rhohat_UB, rhohat_err,
            sigmahat, sigmahat_LB, sigmahat_UB, sigmahat_err,
            X_rmse, Y_rmse, Z_rmse]
    insample_logs.loc[len(insample_logs.index)] = row
    
# save our dataframe to .csv
insample_logs.to_csv("logs/insample_logs.csv", index=False)

  0%|          | 0/320 [00:00<?, ?it/s]

2024-12-20 09:47:13.746404: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-20 09:47:13.782381: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-20 09:47:13.786751: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-20 09:47:13.797458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-20 09:47:13.813834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

  0%|          | 0/1600 [00:00<?, ?it/s]

# Forecasting

In [3]:
# start our forecasting logs (let's go until TM=5.0, only compute on the out-of-sample!)
cols = ["model", "lmbda", "rho", "alpha", "dobs", "seed", "X_rmse", "Y_rmse", "Z_rmse"]
t_max, t_max_pred = 2.0, 5.0

# create our dataframe to store our results
forecast_logs = pd.DataFrame(data=None, columns=cols)

########

# let's start with MAGI first
model = "magi"

# what are our relevant magi files?
fnames = sorted([f for f in os.listdir("results/magi_forecasting") if "rho" in f])

for fname in tqdm(fnames):
    
    # create our row for the dataframe
    rho, alpha, dobs, _, seed = [float(s.split("=")[1]) for s in fname.replace(".pickle", "").split("_")]
    dobs, seed = int(dobs), int(seed)
    row = [model, np.nan, rho, alpha, dobs, seed]
    
    # open up the pickle file
    with open(f"results/magi_forecasting/{fname}", "rb") as file:
        results = pickle.load(file)
    
    # extract our predictions on the desired interval (t_max, t_max_pred]
    preds = pd.DataFrame(
    np.hstack([results[3]["I"], 
               results[3]["X_samps"].mean(axis=0)]), 
    columns=["t", "X", "Y", "Z"]).query(f"t > {t_max} and t <= {t_max_pred}").set_index("t")
    preds.index = np.round(preds.index, 5)
    
    # get our truth
    truth = pd.read_csv(f"data/LORENZ_rho={rho}_alpha={alpha}_seed={seed}.csv")\
    .query(f"t > {t_max} and t <= {t_max_pred}")[["t", "X_true", "Y_true", "Z_true"]].set_index("t")
    truth.index = np.round(truth.index, 5); truth = truth.loc[preds.index]
    
    # compute our RMSEs on the forecasting regions only
    X_rmse, Y_rmse, Z_rmse = np.sqrt(((truth.values - preds.values) ** 2).mean(axis=0))
    
    # add to our row, append to our dataframe
    row += [X_rmse, Y_rmse, Z_rmse]
    forecast_logs.loc[len(forecast_logs.index)] = row
    
########

# proceed to PINN
model = "pinn"

# what are our relevant magi files?
fnames = sorted([f for f in os.listdir("results/pinn_forecasting") if "rho" in f])

for fname in tqdm(fnames):
    
    # create our row for the dataframe
    lmbda, rho, alpha, dobs, seed = [float(s.split("=")[1]) for s in fname.split("_")]
    dobs, seed = int(dobs), int(seed)
    row = [model, lmbda, rho, alpha, dobs, seed]
    
    # extract our predictions on the desired interval (t_max, t_max_pred]
    preds = pd.read_csv(f"results/pinn_forecasting/{fname}/oos_preds.csv")\
    .query(f"t > {t_max} and t <= {t_max_pred}").set_index("t")
    preds.index = np.round(preds.index, 5)
    
    # get our truth
    truth = pd.read_csv(f"data/LORENZ_rho={rho}_alpha={alpha}_seed={seed}.csv")\
    .query(f"t > {t_max} and t <= {t_max_pred}")[["t", "X_true", "Y_true", "Z_true"]].set_index("t")
    truth.index = np.round(truth.index, 5); truth = truth.loc[preds.index]
    
    # compute our RMSEs on the forecasting regions only
    X_rmse, Y_rmse, Z_rmse = np.sqrt(((truth.values - preds.values) ** 2).mean(axis=0))
    
    # add to our row, append to our dataframe
    row += [X_rmse, Y_rmse, Z_rmse]
    forecast_logs.loc[len(forecast_logs.index)] = row
    
# save our dataframe to .csv
forecast_logs.to_csv("logs/forecast_logs.csv", index=False)

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

# Physics

In [ ]:
# model governing the Lorenz system, appropriate for tensorflow vectorization
def f_vec(t, X, thetas):
    '''
    1. X - array containing (X, Y, Z) components. Suppose it is (N x D) for vectorization.
    2. theta - array containing (beta, rho, sigma) components.
    '''
    return tf.concat([thetas[2] * (X[:,1:2] - X[:,0:1]), # dx/dt = sigma * (y-x)
                      X[:,0:1] * (thetas[1] - X[:,2:3]) - X[:,1:2], # dy/dt = x * (rho - z) - y
                      X[:,0:1]*X[:,1:2] - thetas[0]*X[:,2:3], # dz/dt = x*y - beta*z
                     ], axis=1)

# fix our t_max
t_max = 8.0

# create a dataframe to store all of our physics residuals
columns = ["model", "lmbda", "rho", "alpha", "dobs", "seed", "X_pRMSE", "Y_pRMSE", "Z_pRMSE"]
physics_logs = pd.DataFrame(data=None, columns=columns)

# computing physics residuals - let's do separate figures for each seed
for rho in [23.0, 28.0]:
    for alpha in [0.05]:
        for dobs in [10]:
            
            # get the discretization
            discret = int(np.log2(40 // dobs))
            
            # separate figures for each seed!
            for seed in tqdm(range(100)):
                
                # load in our data
                raw_data = pd.read_csv(f"data/LORENZ_rho={rho}_alpha={alpha}_seed={seed}.csv").query(f"t <= {t_max}")
                obs_data = raw_data.iloc[::int((raw_data.index.shape[0] - 1) / (dobs * t_max))]
                ts_obs = obs_data.t.values.astype(np.float64)
                X_obs = obs_data[["X_obs", "Y_obs", "Z_obs"]].to_numpy().astype(np.float64)
                
                ######## START WITH MAGI FIRST! ########
                
                # create our MAGI model + "discretize" our data
                model = magi_v2.MAGI_v2(D_thetas=3, ts_obs=ts_obs, X_obs=X_obs, bandsize=None, f_vec=f_vec)
                I, X_obs_discret = model._discretize(model.ts_obs, model.X_obs, discret)
                X_interp_obs = model._linear_interpolate(X_obs_discret)
                model.mu_ds = X_interp_obs.mean(axis=0)
                
                # load in our MAGI results
                fname = f"results/magi/rho={rho}_alpha={alpha}_dobs={dobs}_discret={discret}_seed={seed}.pickle"
                with open(fname, "rb") as file:
                    results = pickle.load(file)
                    X_samp, theta_samp = results["X_samps"], results["thetas_samps"]
                    
                # update our phi matrices
                model.update_kernel_matrices(
                    I_new=results["I"], 
                    phi1s_new=results["phi1s"], 
                    phi2s_new=results["phi2s"]
                )
                
                #### WORKING ON THE MEAN ####
                
                # now, compute the derivatives using the mean-X/theta
                X_mean, theta_mean = X_samp.mean(axis=0)[None,:], theta_samp.mean(axis=0)[None,:]
                X_cent_mean = tf.reshape(
                    X_mean - model.mu_ds, 
                    shape=(X_mean.shape[0], X_mean.shape[1], 1, X_mean.shape[2])
                )
                f_gp_magi_mean = model.m_ds @ tf.transpose(X_cent_mean, perm=[0, 3, 1, 2])
                f_gp_magi_mean = tf.transpose(f_gp_magi_mean, perm=[0, 2, 1, 3])[:,:,:,0].numpy()
                
                # compute the ODE derivatives on the MAGI samples
                dXdt_ode_magi_mean = theta_mean[:,None,2:3] * (X_mean[:,:,1:2] - X_mean[:,:,0:1])
                dYdt_ode_magi_mean = (X_mean[:,:,0:1] * (theta_mean[:,None,1:2] - X_mean[:,:,2:3])) - X_mean[:,:,1:2]
                dZdt_ode_magi_mean = (X_mean[:,:,0:1] * X_mean[:,:,1:2]) - (theta_mean[:,None,0:1] * X_mean[:,:,2:3])
                f_ode_magi_mean = np.concatenate(
                    [dXdt_ode_magi_mean, 
                     dYdt_ode_magi_mean, 
                     dZdt_ode_magi_mean
                    ], axis=2)
                
                # get the residuals + take their mean
                magi_physics_residuals_mean = np.sqrt(((f_gp_magi_mean - f_ode_magi_mean)[0] ** 2).mean(axis=0))
                
                # add to our row
                row = ["magi", np.nan, rho, alpha, dobs, seed] + list(magi_physics_residuals_mean)
                physics_logs.loc[len(physics_logs.index)] = row
                
                # now go thru all the PINNs
                for i, lmbda in enumerate([0.1, 1.0, 10.0, 100.0, 1000.0]):

                    # load in the physics residuals for this neural network
                    physics_residuals = pd.read_csv(f"results/pinn/lmbda={lmbda}_rho={rho}_" + \
                                                    f"alpha={alpha}_dobs={dobs}_seed={seed}/physics_residuals.csv")
                    physics_residuals = physics_residuals[["X", "Y", "Z"]].to_numpy()
                    physics_residuals_pinn = np.sqrt((physics_residuals ** 2).mean(axis=0))

                    # add to our row
                    row = ["pinn", lmbda, rho, alpha, dobs, seed] + list(physics_residuals_pinn)
                    physics_logs.loc[len(physics_logs.index)] = row
                
# save our physics logs
physics_logs.to_csv("logs/physics_logs.csv", index=False)